In [1]:
import numpy as np
import richdem as rd
import math
import random

In [2]:
import dataFunctions

In [3]:
path_ANH4 = '/Volumes/HydesT7/Grad Project/Data/AHN4_all/'
path_pre = '/Volumes/HydesT7/Grad Project/Data/AHN4_preprocessed_1_8/'

list_patch = []
with open(path_ANH4+'picked.txt') as fp:
    # list_patch.append(patch)
    list_patch = [line.rstrip('\n') for line in fp]
print(list_patch[:5], len(list_patch))

['05FZ1', '05FZ2', '05GN2', '05GZ1', '05GZ2'] 505


In [4]:
cut_width = 64
cut_length = 64

sa = random.choice(list_patch)

dsm30_fill = dataFunctions.readTIFF(path_pre, 'R30_'+sa+'_filled.TIF')
# dsm30_slope = dataFunctions.readTIFF(path_pre, sa)
# dsm30_Roughness = dataFunctions.readTIFF(path_pre, sa)

# dsm_fill_slices = {}

dsm_fill_slices, dt = dataFunctions.sliceRisPic(dsm30_fill, cut_width, cut_length, dt=True)


In [5]:
dt

{0: '0:64,0:64',
 1: '0:64,64:128',
 2: '0:64,103:167',
 3: '64:128,0:64',
 4: '64:128,64:128',
 5: '64:128,103:167',
 6: '128:192,0:64',
 7: '128:192,64:128',
 8: '128:192,103:167',
 9: '144:208,0:64',
 10: '144:208,64:128',
 11: '144:208,103:167'}

In [9]:
dsm30_fill.shape

(208, 167)

In [8]:
dsm_fill_slices

{'0:64,0:64': array([[1.0270172 , 1.0587416 , 1.0936038 , ..., 0.6093449 , 0.6363914 ,
         0.6373706 ],
        [1.0379293 , 1.074853  , 1.0893911 , ..., 0.58602697, 0.6065113 ,
         0.59284675],
        [1.0537839 , 1.0775025 , 1.0787776 , ..., 0.5950348 , 0.59190243,
         0.57559276],
        ...,
        [0.7652866 , 0.79261184, 0.63658756, ..., 0.27957752, 0.32464454,
         0.3214237 ],
        [0.7880441 , 0.88774836, 0.7802002 , ..., 0.33711812, 0.34922525,
         0.338744  ],
        [0.78636545, 0.90496325, 0.88406694, ..., 0.34182325, 0.3707583 ,
         0.3724663 ]], dtype=float32),
 '0:64,64:128': array([[ 0.6408899 ,  0.6887792 ,  0.6057269 , ...,  1.7464572 ,
          2.1825294 ,  2.319233  ],
        [ 0.59562886,  0.63196796,  0.65348834, ...,  6.351863  ,
          4.6653056 ,  2.5845337 ],
        [ 0.56992817,  0.55891347,  0.5878791 , ...,  2.3112283 ,
          2.1981068 ,  3.3306532 ],
        ...,
        [ 0.35654745,  0.3984448 ,  0.39865962,

In [30]:
def readTIFF(path, tiff, masked=False):
    dem = rd.LoadGDAL(path+tiff)
    no_data = dem.no_data
    if not masked:
        return np.asarray(dem)
    elif no_data>0:
        dem_np = np.asarray(dem)
        return np.ma.masked_where(dem_np > no_data/10, dem_np)
    else:
        dem_np = np.asarray(dem)
        return np.ma.masked_where(dem_np < no_data/10, dem_np)

def slicePic(pic, cut_width, cut_length):
    sliced_pic = []
    width = pic.data.shape[0]
    length = pic.data.shape[1]
    num_width = int(width/cut_width)
    num_length = int(length/cut_length)
    for i in range(0, num_width):
        for j in range(0, num_length):
            pic_ = pic[i*cut_width:(i+1)*cut_width, j *
                       cut_length:(j+1)*cut_length]
            sliced_pic.append(pic_)
    return sliced_pic

def sliceRisPic(pic, cut_width, cut_length, dt=False):
    dt_sliced_pic = {}
    ls_sliced_pic = []
    width = pic.data.shape[0]
    length = pic.data.shape[1]
    num_width = math.ceil(width/cut_width)
    num_length = math.ceil(length/cut_length)
    for i in range(0, num_width):
        for j in range(0, num_length):
            pic_ = np.zeros((cut_width, cut_length))
            key_ = ''
            if i <= num_width-2 and j <= num_length-2:
                pic_ = pic[i*cut_width:(i+1)*cut_width, j * cut_length:(j+1)*cut_length]
                key_ = str(i*cut_width)+':'+str((i+1)*cut_width)+','+str(j*cut_length)+':'+str((j+1)*cut_length)
                # sliced_pic[key_]=pic_
            elif i <= num_width-2 and j == num_length-1:
                pic_ = pic[i*cut_width:(i+1)*cut_width, length-cut_length:length]
                key_ = str(i*cut_width)+':'+str((i+1)*cut_width)+','+str(length-cut_length)+':'+str(length)
                # sliced_pic[key_]=pic_
            elif i == num_width-1 and j <= num_length-2:
                pic_ = pic[width-cut_width:width, j * cut_length:(j+1)*cut_length]
                key_ = str(width-cut_width)+':'+str(width)+','+str(j*cut_length)+':'+str((j+1)*cut_length)
            else:
                pic_ = pic[width-cut_width:width, length-cut_length:length]
                key_ = str(width-cut_width)+':'+str(width)+','+str(length-cut_length)+':'+str(length)
                # sliced_pic[key_]=pic_
            # sliced_pic[key_]=pic_
            if not dt:
                ls_sliced_pic.append(pic_)
            else:
                dt_sliced_pic[key_]=pic_
    if not dt:
        return ls_sliced_pic
    else:
        return dt_sliced_pic

In [32]:
path_ANH4 = '/Volumes/HydesT7/Grad Project/Data/AHN4_all/'
path_pre = '/Volumes/HydesT7/Grad Project/Data/AHN4_preprocessed/'

list_patch = []
with open(path_ANH4+'picked.txt') as fp:
        # list_patch.append(patch)
    list_patch = [line.rstrip('\n') for line in fp]

ls_dsm30_fill = []
for name in list_patch:
    ls_dsm30_fill.append('R30_'+name+'_filled.TIF')

In [33]:
import random

sa = random.choice(ls_dsm30_fill)
dem = rd.LoadGDAL(path_pre+sa)
dem = readTIFF(path_pre, sa)
dem.shape

(208, 167)

In [38]:
pics= sliceRisPic(dem, 64, 64, dt=True)

In [39]:
pics

{'0:64,0:64': array([[26.76295 , 23.198761, 19.64183 , ..., 24.355621, 24.302973,
         24.22517 ],
        [24.79364 , 21.332813, 19.657463, ..., 22.918077, 23.387562,
         23.312208],
        [21.875145, 19.429272, 21.491816, ..., 25.111437, 25.054405,
         24.972208],
        ...,
        [25.803394, 24.624954, 26.092066, ..., 18.506653, 18.611998,
         18.623663],
        [26.387709, 25.669949, 25.301437, ..., 18.628077, 18.490118,
         18.451315],
        [25.770525, 26.370544, 25.31153 , ..., 18.561934, 18.476274,
         18.429895]], dtype=float32),
 '0:64,64:128': array([[24.207485, 25.024271, 24.64381 , ..., 26.32096 , 25.06403 ,
         25.06093 ],
        [22.781994, 23.614414, 23.233559, ..., 23.323595, 23.323156,
         23.32275 ],
        [24.4367  , 25.761494, 25.374191, ..., 25.356638, 25.353199,
         25.350014],
        ...,
        [18.5482  , 18.533672, 18.527288, ..., 22.73191 , 22.900934,
         23.00674 ],
        [18.655605, 18.829899

In [41]:
a = np.random.rand(4,4)
b = np.random.rand(2,2)

display(a, b)

array([[0.87669737, 0.10437486, 0.36999713, 0.63894334],
       [0.66544497, 0.9092352 , 0.15517395, 0.33390698],
       [0.9024544 , 0.89993293, 0.98108371, 0.924385  ],
       [0.01394295, 0.01168974, 0.28588365, 0.96627567]])

array([[0.11864817, 0.49600649],
       [0.53331754, 0.34727893]])

In [46]:
a[1:3, 1:3] = np.where(a[1:3, 1:3]<b, a[1:3, 1:3], b)

In [47]:
a

array([[0.87669737, 0.10437486, 0.36999713, 0.63894334],
       [0.66544497, 0.11864817, 0.15517395, 0.33390698],
       [0.9024544 , 0.53331754, 0.34727893, 0.924385  ],
       [0.01394295, 0.01168974, 0.28588365, 0.96627567]])

In [36]:
len(pics)

12

In [14]:
_[0]*64

192

In [10]:
pics = np.asarray(pics)
pics.shape

(6, 64, 64)

In [11]:
data = np.arange(9).reshape((3, 3, 1))

data = data.astype("float")

data[0][0][1] = np.nan

data

array([[[ 0., nan,  2.],
        [ 3.,  4.,  5.],
        [ 6.,  7.,  8.]]])

In [8]:
data = np.arange(9).reshape((3, 3))
data = data.astype("float")
zeros_ = np.zeros(data.shape)

NameError: name 'np' is not defined

In [52]:
data

array([[0., 1., 2.],
       [3., 4., 5.],
       [6., 7., 8.]])

In [45]:
data.shape

(3, 3)

In [46]:
zeros_.shape

(3, 3)

In [55]:
data_ = np.stack((data, zeros_))
data_.shape

(2, 3, 3)

In [56]:
data_

array([[[0., 1., 2.],
        [3., 4., 5.],
        [6., 7., 8.]],

       [[0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.]]])

In [10]:
np.mean(data)

nan

In [12]:
data = np.arange(100)

In [15]:
data[-25:]

array([75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91,
       92, 93, 94, 95, 96, 97, 98, 99])

In [3]:
402*(0.15+0.08+0.02+0.01)

104.51999999999998

In [5]:
403*0.6

241.79999999999998

In [7]:
402*0.15/104

0.5798076923076922